In [1]:
library(data.table)
library(tidyverse)
library(lubridate)
library(Matrix)
# library(slam)
library(bit64)
# library(mtools) for one hot coder, not available on Nero or use caret or tidyr
options(repr.matrix.max.rows=200, repr.matrix.max.cols=30)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.3.3     ✔ purrr   0.3.4
✔ tibble  3.1.2     ✔ dplyr   1.0.6
✔ tidyr   1.1.3     ✔ stringr 1.4.0
✔ readr   1.4.0     ✔ forcats 0.5.1

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::between()   masks data.table::between()
✖ dplyr::filter()    masks stats::filter()
✖ dplyr::first()     masks data.table::first()
✖ dplyr::lag()       masks stats::lag()
✖ dplyr::last()      masks data.table::last()
✖ purrr::transpose() masks data.table::transpose()


Attaching package: ‘lubridate’


The following objects are masked from ‘package:data.table’:

    hour, isoweek, mday, minute, month, quarter, second, wday, week,
    yday, year


The following objects are masked from ‘package:base’:

    date, intersect, setdiff, union



Attaching package: ‘Matrix’


The following objects are masked from ‘package:tidyr’:

    expand, pack, unpack


Loading required package: b

In [1]:
# install.packages(c("dbplyr", "RSQLite"))

Installing packages into ‘/home/jupyter/.R/library’
(as ‘lib’ is unspecified)

also installing the dependency ‘plogr’




In [3]:
datadir = "../../DataTD"
cohortdir = "../../OutputTD/1_cohort"
featuredir = "../../OutputTD/2_features"

In [4]:
adt <- read.csv("../Data/adt_class_serv_loc.csv") 
nrow(adt) 
# sapply(adt, class)
head(adt)

[1] 2711384

,pat_class_c,pat_class,base_pat_class,pat_service,pat_lv_of_care
,<int>,<chr>,<chr>,<chr>,<chr>
1,172,SNF,,Emergency,Acute Care (Assessment or intervention q4-8)
2,166,Bedded Outpatient (corrections only),,Emergency Medicine,
3,166,Bedded Outpatient (corrections only),,Emergency,Acute Care (Assessment or intervention q4-8)
4,166,Bedded Outpatient (corrections only),,Emergency Medicine,
5,174,Outpatient in a Bed,,Emergency,
6,174,Outpatient in a Bed,,Emergency,


In [6]:
adt <- adt %>% mutate(pat_class_c = factor(pat_class_c),
                      pat_class = factor(pat_class),
                      base_pat_class = factor(base_pat_class),
                      pat_service = factor(pat_service),
                      pat_lv_of_care = factor(pat_lv_of_care))
summary(adt)

  pat_class_c                        pat_class          base_pat_class   
 112    :2510276   Emergency Services     :2510276             :1903679  
 128    : 147248   Observation            : 147248   Emergency : 753548  
 126    :  40909   Inpatient              :  40909   Inpatient :  10851  
 140    :   8779   Outpatient             :   8779   Outpatient:  43306  
 122    :   2438   OP Surgery/Procedure   :   2438                       
 171    :   1061   VC OP Surgery/Procedure:   1061                       
 (Other):    673   (Other)                :    673                       
                        pat_service     
 Emergency                    :2280981  
 Emergency Medicine           : 286628  
 Urgent Care                  : 104002  
                              :  27131  
 FINANCIALLY CLEARED          :   3384  
 General Medicine (University):   1587  
 (Other)                      :   7671  
                                                  pat_lv_of_care   
            

In [12]:
adt %>% group_by(pat_class_c, pat_class, base_pat_class, pat_service) %>% 
        count() %>% arrange(pat_class_c, pat_class, -n, pat_service, base_pat_class)

pat_class_c,pat_class,base_pat_class,pat_service,n
<fct>,<fct>,<fct>,<fct>,<int>
112,Emergency Services,,Emergency,1504280
112,Emergency Services,Emergency,Emergency,692933
112,Emergency Services,,Emergency Medicine,162441
112,Emergency Services,,Urgent Care,67424
112,Emergency Services,Emergency,Urgent Care,36578
112,Emergency Services,Emergency,,15703
112,Emergency Services,,,11428
112,Emergency Services,Emergency,Emergency Medicine,7466
112,Emergency Services,,FINANCIALLY CLEARED,3281


In [14]:
adt %>% group_by(pat_class_c, pat_class, pat_service) %>% 
        count() %>% arrange(pat_class_c, pat_class, -n, pat_service)

pat_class_c,pat_class,pat_service,n
<fct>,<fct>,<fct>,<int>
112,Emergency Services,Emergency,2197213
112,Emergency Services,Emergency Medicine,169907
112,Emergency Services,Urgent Care,104002
112,Emergency Services,,27131
112,Emergency Services,FINANCIALLY CLEARED,3384
112,Emergency Services,General Medicine (University),1587
112,Emergency Services,Trauma,1286
112,Emergency Services,Family Medicine,1082
112,Emergency Services,Cardiology,842


In [15]:
adm <- read.csv(file.path(datadir, '1_4_cohort_orderproc_adm.csv'))
nrow(adm)
head(adm)

[1] 112576

,order_type,display_name,description,order_class,order_status,ordering_mode,order_time_jittered_utc,anon_id,pat_enc_csn_id_coded,inpatient_data_id_coded,admit_time,label_max24,label_24hr_recent,admit_label,has_admit_label,died_within_24hrs,death_24hr_max_label,death_24hr_recent_label,first_label,first_label_minutes_since_admit,acute_to_critical_label_recent,critical_to_acute_label_recent,acute_to_critical_label_max,critical_to_acute_label_max
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<int>,<chr>,<int>,<int>,<dbl>,<int>,<int>,<int>,<int>,<int>,<dbl>,<int>,<int>,<int>,<int>
1,Admission,Admit to Inpatient,ADMIT TO INPATIENT,Hospital Performed,Completed,Inpatient,2017-08-13 01:17:00+00:00,JCcead62,131236952067,25613940,2017-08-13 01:17:00+00:00,0,0,0,1,0,0,0,0,0,0,0,0,0
2,Admission,Admit to Inpatient,ADMIT TO INPATIENT,Hospital Performed,Completed,Inpatient,2018-08-19 06:56:00+00:00,JCe97cc2,131254643960,31279081,2018-08-19 06:56:00+00:00,1,1,1,1,0,1,1,1,0,0,0,0,0
3,Admission,Admit to Inpatient: Expected stay >2 midnights,ADMIT TO INPATIENT,Hospital Performed,Completed,Inpatient,2017-03-03 02:38:00+00:00,JCe2b4c2,131218333748,23455975,2017-03-03 02:38:00+00:00,0,0,0,1,0,0,0,0,0,0,0,0,0
4,Admission,Admit to Inpatient,ADMIT TO INPATIENT,Hospital Performed,Completed,Inpatient,2017-08-31 17:51:00+00:00,JCe75254,131237208654,25711443,2017-08-31 17:51:00+00:00,0,0,0,1,0,0,0,0,0,0,0,0,0
5,Admission,Admit to Inpatient: Expected stay >2 midnights,ADMIT TO INPATIENT,Hospital Performed,Completed,Inpatient,2017-04-10 19:57:00+00:00,JCeab009,131226874377,24366937,2017-04-10 19:57:00+00:00,0,0,0,1,0,0,0,0,0,0,0,0,0
6,Admission,Admit to Inpatient: Expected stay >2 midnights,ADMIT TO INPATIENT,Hospital Performed,Completed,Inpatient,2016-04-10 11:46:00+00:00,JCd314d5,131184406356,19500578,2016-04-10 11:46:00+00:00,0,0,0,1,0,0,0,0,0,0,0,0,0


In [14]:
adm %>% group_by(description, display_name, order) %>% count(sort=TRUE)

description,display_name,n
<chr>,<chr>,<int>
ADMIT TO INPATIENT,Admit to Inpatient,83581
ADMIT TO INPATIENT,Admit to Inpatient: Expected stay >2 midnights,22504
CONVERT INPATIENT TO OBSERVATION,Convert Inpatient to Observation per Case Management Review,2462
PLACE IN OBSERVATION,Place in Observation,1890
PLACE IN OUTPATIENT PROCEDURE/PRE-OP AND POST-PROCEDURE RECOVERY,Place in Outpatient Procedure/Pre-Op and Post-procedure Recovery,663
ADMIT TO INPATIENT,Admit to Inpatient: Expected stay >2 midnights,586
PLACE IN OBSERVATION,Place in Observation: Expected stay <2 midnights,324
PLACE IN OBSERVATION,Place in Observation-CDU,310
ADMIT TO SKILLED NURSING FACILITY,Admit to Skilled Nursing Facility,182


In [ ]:
# take all the order within 24 hours after admit_time
# remove patients further